# Health Information portal

## Libraries

In [ ]:
from rdflib import Graph, URIRef, BNode, Literal, Namespace
import requests, os, pathlib
from pathlib import Path

### Get data catalogs in an FDP

In [ ]:
# tbd

## Get data catalog

e.g. found uuid for Belgian Node here https://fair.healthinformationportal.eu/

In [ ]:
mycatalog = '743a4ddb-b3b3-4c78-8bad-ab8d7d9b0f51'
myfdp = 'https://fair.healthinformationportal.eu/catalog/'
uribe = myfdp+mycatalog+'/'

In [ ]:
mysyntax = 'text/turtle'
#mysyntax = 'application/ld+json'
#mysyntax = 'application/rdf+xml'
#mysyntax = 'text/n3'
headers = {'Accept': mysyntax}

In [ ]:
resbe = requests.get(url=uribe, headers=headers)
print(resbe.text)

## Get Data Catalog and its children

https://fair.healthinformationportal.eu/swagger-ui/index.html#/Metadata%3A%20Catalog/getCatalogExpanded

In [ ]:
uribeexp = uribe + 'expanded'

In [ ]:
mysyntax = 'text/turtle'
#mysyntax = 'application/ld+json'
#mysyntax = 'application/rdf+xml'
#mysyntax = 'text/n3'
headers = {'Accept': mysyntax}

In [ ]:
resbe = requests.get(url=uribeexp, headers=headers)

## Turn to graph

In [ ]:
resbe_gr = Graph().parse(data=resbe.text)

## Enumerate datasets via iterator

In [ ]:
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, VOID, XMLNS, XSD

In [ ]:
#for s,p,o in resbe_gr:
  #print(s+"==="+p+"==="+o)

In [ ]:
dbbe_list = []
for s,p,o in resbe_gr.triples((None,DCAT['dataset'],None)):
  #print(o)
  dbbe_list.append(o)

In [ ]:
dbbe_list

In [ ]:
dbbe_list_e = [db[:].split('/')[-1] for db in dbbe_list]   # keep only uuid, last part of url

Create directories

In [ ]:
for duuid in dbbe_list_e:
    Path('../opendistributions/'+mycatalog+'/'+duuid).mkdir(parents=True, exist_ok=True)

    f = open('../opendistributions/'+mycatalog+'/'+duuid+"/readme.txt", "a")
    f.write("A place to include open distributions content!")
    f.close()

## Enumerate datasets via query

In [ ]:
datasets_query = """
SELECT DISTINCT ?o
WHERE {
  ?s dcat:dataset ?o .
}"""

In [ ]:
qres = resbe_gr.query(datasets_query)

In [ ]:
dbbe_list_m2 = []
for row in qres:
  dbbe_list_m2.append(row.o)

In [ ]:
dbbe_list_m2

## Query datasets
(restricted to three here)

In [ ]:
mydb = dbbe_list_m2[0:3]

In [ ]:
mydb

In [ ]:
res_all = Graph()
for urin in mydb:

  res = requests.get(url=urin, headers=headers)
  gres = Graph().parse(data=res.text)

  res_all = res_all + gres

In [ ]:
#print(res_all.serialize())

### Make dir structure